In [5]:
import pandas as pd
import json

# Load JSON data from files
def load_json(file_path):
    with open(file_path) as f:
        return json.load(f)

In [6]:
# Function to load JSON from a saved file
def load_json_from_file(filename):
    with open(filename, 'r') as f:
        return json.load(f)

In [10]:
# Step 2: Save uploaded brands.json to disk
brands_filename = list(brands_uploaded.keys())[0]

In [11]:
# Function to load multiple JSON objects (line-separated) from a file
def load_json_objects_from_file(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Step 1: Upload brands.json
print("Please upload the brands.json file.")
brands_uploaded = files.upload()  # Upload brands.json

# Step 2: Save uploaded brands.json to disk
brands_filename = list(brands_uploaded.keys())[0]

# Step 3: Load the JSON objects from the saved file
brands_data = load_json_objects_from_file(brands_filename)

Please upload the brands.json file.


Saving brands.json to brands (1).json


In [12]:
# Step 4: Upload users.json
print("Please upload the users.json file.")
users_uploaded = files.upload()  # Upload users.json
users_filename = list(users_uploaded.keys())[0]
users_data = load_json_objects_from_file(users_filename)

Please upload the users.json file.


Saving users.json to users.json


In [13]:
# Step 5: Upload receipts.json
print("Please upload the receipts.json file.")
receipts_uploaded = files.upload()  # Upload receipts.json
receipts_filename = list(receipts_uploaded.keys())[0]
receipts_data = load_json_objects_from_file(receipts_filename)

Please upload the receipts.json file.


Saving receipts.json to receipts.json


In [17]:
# Step 6: Flatten nested lists like 'rewardsReceiptItemList' and convert to DataFrame

# Normalize receipts that have 'rewardsReceiptItemList'
receipts_with_items = [receipt for receipt in receipts_data if 'rewardsReceiptItemList' in receipt]

# If receipts with items exist, normalize them
if receipts_with_items:
    receipts_df = pd.json_normalize(
        receipts_with_items,
        record_path='rewardsReceiptItemList',
        meta=['_id', 'bonusPointsEarned', 'createDate', 'rewardsReceiptStatus', 'totalSpent', 'userId'],
        errors='ignore'
    )
else:
    # If no nested items are found, just flatten the top-level structure
    receipts_df = pd.json_normalize(receipts_data, errors='ignore')

# Display the receipts DataFrame
print("Receipts DataFrame:")
print(receipts_df.head())


Receipts DataFrame:
        barcode                                        description finalPrice  \
0          4011                                     ITEM NOT FOUND      26.00   
1          4011                                     ITEM NOT FOUND          1   
2  028400642255  DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...      10.00   
3           NaN                                                NaN        NaN   
4          4011                                     ITEM NOT FOUND      28.00   

  itemPrice needsFetchReview partnerItemId preventTargetGapPoints  \
0     26.00            False             1                   True   
1         1              NaN             1                    NaN   
2     10.00             True             2                   True   
3       NaN            False             1                   True   
4     28.00            False             1                   True   

   quantityPurchased userFlaggedBarcode userFlaggedNewItem  ...  \
0          

In [19]:
# Convert JSON data to DataFrames
brands_df = pd.json_normalize(brands_data)

# Handling missing fields by providing a flexible normalization of the JSON structure
users_df = pd.json_normalize(users_data)

In [20]:
print(users_df.head())
print(brands_df.head())

   active      role signUpSource state                  _id.$oid  \
0    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   
1    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   
2    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   
3    True  consumer        Email    WI  5ff1e1eacfcf6c399c274ae6   
4    True  consumer        Email    WI  5ff1e194b6a9d73a3a9f1052   

   createdDate.$date  lastLogin.$date  
0      1609687444800     1.609688e+12  
1      1609687444800     1.609688e+12  
2      1609687444800     1.609688e+12  
3      1609687530554     1.609688e+12  
4      1609687444800     1.609688e+12  
        barcode        category      categoryCode                       name  \
0  511111019862          Baking            BAKING  test brand @1612366101024   
1  511111519928       Beverages         BEVERAGES                  Starbucks   
2  511111819905          Baking            BAKING  test brand @1612366146176   
3  511111519874          Baking

In [24]:
# Inspecting the structure of both DataFrames to understand potential issues
print("Brands DataFrame Structure:")
print(brands_df.info())

print("\nUsers DataFrame Structure:")
print(users_df.info())

# Checking for missing values to identify incomplete records
print("\nMissing values in the 'brands_df':")
print(brands_df.isnull().sum())

print("\nMissing values in the 'users_df':")
print(users_df.isnull().sum())

# Checking for duplicates in 'id_oid' which might serve as primary keys
print("\nChecking for duplicate entries in '_id.$oid' in brands_df:")
duplicate_brands = brands_df[brands_df.duplicated(subset='_id.$oid', keep=False)]
print(duplicate_brands)

print("\nChecking for duplicate entries in '_id.$oid' in users_df:")
duplicate_users = users_df[users_df.duplicated(subset='_id.$oid', keep=False)]
print(duplicate_users)

# Inspecting unique values of critical fields to understand the range of values
print("\nUnique values for 'active' in users_df:")
print(users_df['active'].unique())

print("\nUnique values for 'topBrand' in brands_df:")
print(brands_df['topBrand'].unique())


Brands DataFrame Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   object
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      555 non-null    object
 5   _id.$oid      1167 non-null   object
 6   cpg.$id.$oid  1167 non-null   object
 7   cpg.$ref      1167 non-null   object
 8   brandCode     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB
None

Users DataFrame Structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   active             495 non-null    bool   
 1   role               495 non-null    object 
 2   signUpSource       447 non-nu

In [28]:
# Checking missing values across key fields
print("\nChecking for missing values in critical fields:")
print("Brands DataFrame:")
print(brands_df[['_id.$oid', 'barcode', 'name']].isnull().sum())

print("\nUsers DataFrame:")
# Correct the field names for 'createdDate' and 'lastLogin'
print(users_df[['_id.$oid', 'active', 'createdDate.$date']].isnull().sum())

# Checking for potential outliers in numeric or date fields (such as future dates)
print("\nChecking for future or invalid 'createdDate.$date' in users_df:")
from datetime import datetime
current_date = datetime.now()

# Convert 'createdDate.$date' to a datetime format for comparison
users_df['createdDate.$date'] = pd.to_datetime(users_df['createdDate.$date'], unit='ms', errors='coerce')

# Check for future dates
future_dates = users_df[users_df['createdDate.$date'] > current_date]
print(future_dates)

# Checking for inconsistent or out-of-range values in the 'active' field
print("\nReviewing inconsistent 'active' values in users_df:")
print(users_df['active'].value_counts())



Checking for missing values in critical fields:
Brands DataFrame:
_id.$oid    0
barcode     0
name        0
dtype: int64

Users DataFrame:
_id.$oid             0
active               0
createdDate.$date    0
dtype: int64

Checking for future or invalid 'createdDate.$date' in users_df:
Empty DataFrame
Columns: [active, role, signUpSource, state, _id.$oid, createdDate.$date, lastLogin.$date]
Index: []

Reviewing inconsistent 'active' values in users_df:
active
True     494
False      1
Name: count, dtype: int64


In [31]:
# Handling missing values and deciding what to do with them
# Dropping rows with missing critical fields like 'barcode'
print("\nDropping rows with missing 'barcode' in brands_df:")
cleaned_brands_df = brands_df.dropna(subset=['barcode'])
print(cleaned_brands_df.info())

# Identifying and handling duplicate records in '_id.$oid'
print("\nHandling duplicate '_id.$oid' in users_df:")
cleaned_users_df = users_df.drop_duplicates(subset='_id.$oid')
print(cleaned_users_df.info())

# Investigating invalid or future 'createdDate.$date'
print("\nFiltering out users with future 'createdDate.$date':")

# Convert 'createdDate.$date' to datetime if not already done
if cleaned_users_df['createdDate.$date'].dtype != 'datetime64[ns]':
    cleaned_users_df['createdDate.$date'] = pd.to_datetime(cleaned_users_df['createdDate.$date'], unit='ms', errors='coerce')

# Filter out rows where 'createdDate.$date' is in the future
valid_users_df = cleaned_users_df[cleaned_users_df['createdDate.$date'] <= current_date]
print(valid_users_df.info())



Dropping rows with missing 'barcode' in brands_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   object
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      555 non-null    object
 5   _id.$oid      1167 non-null   object
 6   cpg.$id.$oid  1167 non-null   object
 7   cpg.$ref      1167 non-null   object
 8   brandCode     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB
None

Handling duplicate '_id.$oid' in users_df:
<class 'pandas.core.frame.DataFrame'>
Index: 212 entries, 0 to 475
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   active             212 non-null    bool          
 1   role               21

In [33]:
# Investigating the unique categories in the 'categoryCode' field to understand the categorization of brands
print("\nUnique values in 'categoryCode' in brands_df:")
print(brands_df['categoryCode'].unique())

# Reviewing the distribution of 'createdDate.$date' to understand user sign-up patterns
print("\nDistribution of 'createdDate.$date' in users_df:")
print(users_df['createdDate.$date'].describe())



Unique values in 'categoryCode' in brands_df:
['BAKING' 'BEVERAGES' 'CANDY_AND_SWEETS' nan 'HEALTHY_AND_WELLNESS'
 'GROCERY' 'PERSONAL_CARE' 'CLEANING_AND_HOME_IMPROVEMENT'
 'BEER_WINE_SPIRITS' 'BABY' 'BREAD_AND_BAKERY' 'OUTDOOR'
 'DAIRY_AND_REFRIGERATED' 'MAGAZINES' 'FROZEN']

Distribution of 'createdDate.$date' in users_df:
count                              495
mean     2020-08-06 01:34:47.878830592
min         2014-12-19 14:21:22.381000
25%      2021-01-04 19:30:17.483500032
50%      2021-01-13 20:19:38.720999936
75%      2021-01-25 17:31:59.408999936
max         2021-02-12 14:11:06.240000
Name: createdDate.$date, dtype: object


In [34]:
# Checking the size of the dataframes to anticipate performance bottlenecks
print("Memory usage of brands_df:")
print(brands_df.memory_usage(deep=True))

print("\nMemory usage of users_df:")
print(users_df.memory_usage(deep=True))

# Suggesting the use of efficient file formats and compression to reduce data size
print("\nSaving brands_df to a compressed Parquet file for efficient storage:")
cleaned_brands_df.to_parquet('brands_data.parquet', compression='gzip')

print("\nSaving users_df to a compressed Parquet file:")
cleaned_users_df.to_parquet('users_data.parquet', compression='gzip')


Memory usage of brands_df:
Index             128
barcode         80523
category        72524
categoryCode    54918
name            90592
topBrand        37468
_id.$oid        94527
cpg.$id.$oid    94527
cpg.$ref        71187
brandCode       77977
dtype: int64

Memory usage of users_df:
Index                  128
active                 495
role                 32421
signUpSource         29254
state                27693
_id.$oid             40095
createdDate.$date     3960
lastLogin.$date       3960
dtype: int64

Saving brands_df to a compressed Parquet file for efficient storage:

Saving users_df to a compressed Parquet file:
